<a href="https://colab.research.google.com/github/MariaPosadaBotero/Entregas-del-proyecto-de-AI-para-ingenieros---Udea-2025-2/blob/main/03-modelo_con_preprocesado_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MODELO XGBoost con pipeline


In [1]:
#1. IMPORTAR LAS LIBRERIAS
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report


In [2]:
# 2. CARGAR DATOS CRUDOS

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

print(train.shape, test.shape)
print(train.head())

(692500, 21) (296786, 20)
       ID  PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0  904256              20212                  ENFERMERIA              BOGOTÁ   
1  645256              20212                     DERECHO           ATLANTICO   
2  308367              20203       MERCADEO Y PUBLICIDAD              BOGOTÁ   
3  470353              20195  ADMINISTRACION DE EMPRESAS           SANTANDER   
4  989032              20212                  PSICOLOGIA           ANTIOQUIA   

                E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones    Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                    0   
2  Entre 2.5 millones y menos de 4 millones      Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                    0   
4  Entre 2.5 millones y menos de 4 millones  Entre 21 y 30 horas   

  F_ESTRATOVIVIENDA F_TIENEINTERNET                    F_EDUCACIONPADRE  \
0        

In [3]:
# 3. CREAR TARGET
train = train.rename(columns={"RENDIMIENTO_GLOBAL": "target"})


In [4]:
# 4. SEPARAR X e Y
X = train.drop(columns=["target"])
y = train["target"]


In [5]:
# 5. DIVIDIR EN TRAIN / VALIDACIÓN ( como el profe dijo)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
# 6. IDENTIFICAR COLUMNAS NUMÉRICAS Y CATEGÓRICAS

num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X_train.select_dtypes(include=["object"]).columns

print("Numéricas:", len(num_cols))
print("Categóricas:", len(cat_cols))


Numéricas: 6
Categóricas: 14


In [7]:
# 7. PREPROCESAMIENTO AUTOMÁTICO

preprocesamiento = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_cols)
])

In [8]:
# 8. PIPELINE PARA EL PROCESAMIENTO Y MODELADO FINAL (AUTOCONTENIDO)

modelo = Pipeline([
    ("prep", preprocesamiento),
    ("clf", LinearSVC(C=1.0, max_iter=8000, dual=False))
])


In [9]:
# 9. ENTRENAR

modelo.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['ID', 'PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3',
       'INDICADOR_4'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHo...
                                                  Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE'],
      dtype='object'))])),
                ('clf', LinearSVC(dual=False, max_iter=8000))])

In [10]:
# Accuracy en entrenamiento
from sklearn.metrics import accuracy_score

y_train_pred = modelo.predict(X_train)
accuracy_train = accuracy_score(y_train, y_train_pred)

# Accuracy en validación
y_val_pred = modelo.predict(X_val)
accuracy_val = accuracy_score(y_val, y_val_pred)

print("Accuracy en entrenamiento:", accuracy_train)
print("Accuracy en validación:", accuracy_val)

Accuracy en entrenamiento: 0.42840974729241876
Accuracy en validación: 0.4230902527075812


In [11]:
# 10. VALIDAR

pred_val = modelo.predict(X_val)
print("\nAccuracy en validación:", accuracy_score(y_val, pred_val))
print("\nReporte de clasificación:")
print(classification_report(y_val, pred_val))


Accuracy en validación: 0.4230902527075812

Reporte de clasificación:
              precision    recall  f1-score   support

        alto       0.50      0.67      0.57     35124
        bajo       0.43      0.60      0.50     34597
  medio-alto       0.32      0.21      0.25     34324
  medio-bajo       0.33      0.22      0.26     34455

    accuracy                           0.42    138500
   macro avg       0.40      0.42      0.40    138500
weighted avg       0.40      0.42      0.40    138500



In [12]:
# 11. PREDICCIONES PARA KAGGLE

pred_test = modelo.predict(test)

submission = pd.DataFrame({
    "id": np.arange(len(pred_test)),
    "target": pred_test
})

submission.to_csv("submission.csv", index=False)
print("\nArchivo submission.csv creado correctamente.")


Archivo submission.csv creado correctamente.
